In [ ]:
example = """
....#.....
.........#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#...
""".strip().splitlines()

In [ ]:
import numpy as np

def read(input):
    return np.array([[char for char in line] for line in input])

In [ ]:
def simulate(data):
    done = False
    while not done:
        # print(data)
        gy, gx = np.where(data == "^")
        if gy.shape[0] and gx.shape[0]:
            gy = gy[0]
            gx = gx[0]
            pos = np.where(data[:, gx] == "#")[0]
            i = 0
            pos = list(reversed(pos))
            while i < len(pos) and pos[i] > gy:
                i += 1
            if i == len(pos):
                data[:gy+1, gx] = "X"
                done = True
            else:
                data[pos[i]+1:gy+1, gx] = "X"
                data[pos[i]+1, gx] = ">"
            
            continue
        
        gy, gx = np.where(data == ">")
        if gy.shape[0] and gx.shape[0]:
            gy = gy[0]
            gx = gx[0]
            pos = np.where(data[gy, :] == "#")[0]
            i = 0
            while i < len(pos) and pos[i] < gx:
                i += 1
            if i == len(pos):
                data[gy, gx:] = "X"
                done = True
            else:
                data[gy, gx:pos[i]] = "X"
                data[gy, pos[i]-1] = "v"
            
            continue
        
        gy, gx = np.where(data == "v")
        if gy.shape[0] and gx.shape[0]:
            gy = gy[0]
            gx = gx[0]
            pos = np.where(data[:, gx] == "#")[0]
            i = 0
            while i < len(pos) and pos[i] < gy:
                i += 1
            if i == len(pos):
                data[gy:, gx] = "X"
                done = True
            else:
                data[gy:pos[i], gx] = "X"
                data[pos[i]-1, gx] = "<"
            
            continue

        gy, gx = np.where(data == "<")
        if gy.shape[0] and gx.shape[0]:
            gy = gy[0]
            gx = gx[0]
            pos = np.where(data[gy, :] == "#")[0]
            i = 0
            pos = list(reversed(pos))
            while i < len(pos) and pos[i] > gx:
                i += 1
            if i == len(pos):
                data[gy, :gx+1] = "X"
                done = True
            else:
                data[gy, pos[i]+1:gx+1] = "X"
                data[gy, pos[i]+1] = "^"
            
            continue
    
    return np.count_nonzero(data == "X")

In [ ]:
data = read(example)
simulate(data)

In [ ]:
from pathlib import Path

input = Path("1.txt").read_text().splitlines()

In [ ]:
simulate(read(input))

In [ ]:
def is_loop(data):
    for _ in range(1_000):
        # print(data)
        gy, gx = np.where(data == "^")
        if gy.shape[0] and gx.shape[0]:
            gy = gy[0]
            gx = gx[0]
            pos = np.where(data[:, gx] == "#")[0]
            i = 0
            pos = list(reversed(pos))
            while i < len(pos) and pos[i] > gy:
                i += 1
            if i == len(pos):
                data[:gy+1, gx] = "X"
                return False
            else:
                # if data[pos[i]+1:gy, gx].shape != (0,) and np.all(data[pos[i]+1:gy, gx] == "X"):
                #     return True
                data[pos[i]+1:gy+1, gx] = "X"
                data[pos[i]+1, gx] = ">"
            
            continue
        
        gy, gx = np.where(data == ">")
        if gy.shape[0] and gx.shape[0]:
            gy = gy[0]
            gx = gx[0]
            pos = np.where(data[gy, :] == "#")[0]
            i = 0
            while i < len(pos) and pos[i] < gx:
                i += 1
            if i == len(pos):
                data[gy, gx:] = "X"
                return False
            else:
                # if data[gy, gx+1:pos[i]].shape != (0,) and np.all(data[gy, gx+1:pos[i]] == "X"):
                #     return True
                data[gy, gx:pos[i]] = "X"
                data[gy, pos[i]-1] = "v"
            
            continue
        
        gy, gx = np.where(data == "v")
        if gy.shape[0] and gx.shape[0]:
            gy = gy[0]
            gx = gx[0]
            pos = np.where(data[:, gx] == "#")[0]
            i = 0
            while i < len(pos) and pos[i] < gy:
                i += 1
            if i == len(pos):
                data[gy:, gx] = "X"
                return False
            else:
                # if data[gy+1:pos[i], gx].shape != (0,) and np.all(data[gy+1:pos[i], gx] == "X"):
                #     return True
                data[gy:pos[i], gx] = "X"
                data[pos[i]-1, gx] = "<"
            
            continue

        gy, gx = np.where(data == "<")
        if gy.shape[0] and gx.shape[0]:
            gy = gy[0]
            gx = gx[0]
            pos = np.where(data[gy, :] == "#")[0]
            i = 0
            pos = list(reversed(pos))
            while i < len(pos) and pos[i] > gx:
                i += 1
            if i == len(pos):
                data[gy, :gx+1] = "X"
                return False
            else:
                # if data[gy, pos[i]+1:gx].shape != (0,) and np.all(data[gy, pos[i]+1:gx] == "X"):
                #     return True
                data[gy, pos[i]+1:gx+1] = "X"
                data[gy, pos[i]+1] = "^"
            
            continue
    
    return True


In [ ]:
import copy


def count_loops(data):
    s = 0

    data_intersect = copy.deepcopy(data)
    simulate(data_intersect)

    for y, x in zip(*np.where(data_intersect == "X")):
        if data[y, x] != ".":
            continue
        data_copy = copy.deepcopy(data)
        data_copy[y, x] = "#"
        if is_loop(data_copy):
            s += 1

            # debug
            # data_copy[y, x] = "O"
            # print(data_copy)
    return s

In [ ]:
count_loops(read(example))

In [ ]:
count_loops(read(input))